In [1]:
import pandas as pd
import numpy as np
import gc
import pickle
import psutil
import joblib

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import sqlite3

In [2]:
c1, c1_2, c2, c3 , c4 = 0.175, 0.075, 0.25, 0.25, 0.25

## SAINT+

In [3]:
MAX_SEQ = 100
n_part = 7
D_MODEL = 256
N_LAYER = 2
DROPOUT = 0.1

In [4]:
def feature_time_lag(df, time_dict):

    tt = np.zeros(len(df), dtype=np.int64)

    for ind, row in enumerate(df[['user_id','timestamp','task_container_id']].values):

        if row[0] in time_dict.keys():
            if row[2]-time_dict[row[0]][1] == 0:

                tt[ind] = time_dict[row[0]][2]

            else:
                t_last = time_dict[row[0]][0]
                task_ind_last = time_dict[row[0]][1]
                tt[ind] = row[1]-t_last
                time_dict[row[0]] = (row[1], row[2], tt[ind])
        else:
            # time_dict : timestamp, task_container_id, lag_time
            time_dict[row[0]] = (row[1], row[2], -1)
            tt[ind] =  0

    df["time_lag"] = tt
    return df

In [5]:
class FFN(nn.Module):
    def __init__(self, state_size=200):
        super(FFN, self).__init__()
        self.state_size = state_size

        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.dropout = nn.Dropout(DROPOUT)
    
    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return self.dropout(x)

def future_mask(seq_length):
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)


class SAINTModel(nn.Module):
    def __init__(self, n_skill, n_part, max_seq=MAX_SEQ, embed_dim= 128, elapsed_time_cat_flag = True):
        super(SAINTModel, self).__init__()

        self.n_skill = n_skill
        self.embed_dim = embed_dim
        self.n_cat = n_part
        self.elapsed_time_cat_flag = elapsed_time_cat_flag

        self.e_embedding = nn.Embedding(self.n_skill+1, embed_dim) ## exercise
        self.c_embedding = nn.Embedding(self.n_cat+1, embed_dim) ## category
        self.pos_embedding = nn.Embedding(max_seq-1, embed_dim) ## position
        self.res_embedding = nn.Embedding(2+1, embed_dim) ## response


        if self.elapsed_time_cat_flag == True:
            self.elapsed_time_embedding = nn.Embedding(300+1, embed_dim) ## elapsed time (the maximum elasped time is 300)
            self.lag_embedding1 = nn.Embedding(300+1, embed_dim) ## lag time1 for 300 seconds
            self.lag_embedding2 = nn.Embedding(1440+1, embed_dim) ## lag time2 for 1440 minutes
            self.lag_embedding3 = nn.Embedding(365+1, embed_dim) ## lag time3 for 365 days

        else:
            self.elapsed_time_embedding = nn.Linear(1, embed_dim, bias=False) ## elapsed time
            self.lag_embedding = nn.Linear(1, embed_dim, bias=False) ## lag time


        self.exp_embedding = nn.Embedding(2+1, embed_dim) ## user had explain

        self.transformer = nn.Transformer(nhead=8, d_model = embed_dim, num_encoder_layers= N_LAYER, num_decoder_layers= N_LAYER, dropout = DROPOUT)

        self.dropout = nn.Dropout(DROPOUT)
        self.layer_normal = nn.LayerNorm(embed_dim) 
        self.ffn = FFN(embed_dim)
        self.pred = nn.Linear(embed_dim, 1)
    
    def forward(self, question, part, response, elapsed_time, lag_time, exp):

        device = question.device  

        ## embedding layer
        question = self.e_embedding(question)
        part = self.c_embedding(part)
        pos_id = torch.arange(question.size(1)).unsqueeze(0).to(device)
        pos_id = self.pos_embedding(pos_id)
        res = self.res_embedding(response)
        exp = self.exp_embedding(exp)

        if self.elapsed_time_cat_flag == True:

            ## feature engineering
            ## elasped time
            elapsed_time = torch.true_divide(elapsed_time, 1000)
            elapsed_time = torch.round(elapsed_time)
            elapsed_time = torch.where(elapsed_time.float() <= 300, elapsed_time, torch.tensor(300.0).to(device)).long()
            elapsed_time = self.elapsed_time_embedding(elapsed_time)

            ## lag_time1
            lag_time = torch.true_divide(lag_time, 1000)
            lag_time = torch.round(lag_time)
            lag_time1 = torch.where(lag_time.float() <= 300, lag_time, torch.tensor(300.0).to(device)).long()

            ## lag_time2
            lag_time = torch.true_divide(lag_time, 60)
            lag_time = torch.round(lag_time)
            lag_time2 = torch.where(lag_time.float() <= 1440, lag_time, torch.tensor(1440.0).to(device)).long()

            ## lag_time3
            lag_time = torch.true_divide(lag_time, 1440)
            lag_time = torch.round(lag_time)
            lag_time3 = torch.where(lag_time.float() <= 365, lag_time, torch.tensor(365.0).to(device)).long()

            ## lag time
            lag_time1 = self.lag_embedding1(lag_time1) 
            lag_time2 = self.lag_embedding2(lag_time2) 
            lag_time3 = self.lag_embedding3(lag_time3)

        else:

            elapsed_time = elapsed_time.view(-1,1)
            elapsed_time = self.elapsed_time_embedding(elapsed_time)
            elapsed_time = elapsed_time.view(-1, MAX_SEQ-1, self.embed_dim)

            lag_time = lag_time.view(-1,1)
            lag_time = self.lag_embedding(lag_time)
            lag_time = lag_time.view(-1, MAX_SEQ-1, self.embed_dim)

            # elapsed_time = elapsed_time.view(-1, MAX_SEQ-1, 1)  ## [batch, s_len] => [batch, s_len, 1]
            # elapsed_time = self.elapsed_time_embedding(elapsed_time)


        enc = question + part + pos_id + exp
        dec = pos_id + res + elapsed_time + lag_time1 + lag_time2 + lag_time3

        enc = enc.permute(1, 0, 2) # x: [bs, s_len, embed] => [s_len, bs, embed]
        dec = dec.permute(1, 0, 2)
        mask = future_mask(enc.size(0)).to(device)

        att_output = self.transformer(enc, dec, src_mask=mask, tgt_mask=mask, memory_mask = mask)
        att_output = self.layer_normal(att_output)
        att_output = att_output.permute(1, 0, 2) # att_output: [s_len, bs, embed] => [bs, s_len, embed]

        x = self.ffn(att_output)
        x = self.layer_normal(x + att_output)
        x = self.pred(x)

        return x.squeeze(-1)

In [6]:
n_skill = 13523
group = joblib.load("../input/saint-plus-data-new/group_20210102.pkl.zip")
questions_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
time_dict = joblib.load("../input/saint-plus-data-new/time_dict.pkl.zip")

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model1 = SAINTModel(n_skill, n_part, embed_dim= D_MODEL)
try:
    model1.load_state_dict(torch.load("../input/saint-plus-model/saint_plus_model_20210102_padding_v2.pt"))
except:
    model1.load_state_dict(torch.load("../input/saint-plus-model/saint_plus_model_20210102_padding_v2.pt", map_location='cpu'))
model1.to(device)
model1.eval()

SAINTModel(
  (e_embedding): Embedding(13524, 256)
  (c_embedding): Embedding(8, 256)
  (pos_embedding): Embedding(99, 256)
  (res_embedding): Embedding(3, 256)
  (elapsed_time_embedding): Embedding(301, 256)
  (lag_embedding1): Embedding(301, 256)
  (lag_embedding2): Embedding(1441, 256)
  (lag_embedding3): Embedding(366, 256)
  (exp_embedding): Embedding(3, 256)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dro

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model3 = SAINTModel(n_skill, n_part, embed_dim= D_MODEL)
try:
    model3.load_state_dict(torch.load("../input/saint-plus-model/saint_plus_model_20210107_v3.pt"))
except:
    model3.load_state_dict(torch.load("../input/saint-plus-model/saint_plus_model_20210107_v3.pt", map_location='cpu'))
model3.to(device)
model3.eval()

SAINTModel(
  (e_embedding): Embedding(13524, 256)
  (c_embedding): Embedding(8, 256)
  (pos_embedding): Embedding(99, 256)
  (res_embedding): Embedding(3, 256)
  (elapsed_time_embedding): Embedding(301, 256)
  (lag_embedding1): Embedding(301, 256)
  (lag_embedding2): Embedding(1441, 256)
  (lag_embedding3): Embedding(366, 256)
  (exp_embedding): Embedding(3, 256)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dro

In [9]:
N_LAYER = 3

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model2 = SAINTModel(n_skill, n_part, embed_dim= D_MODEL)
try:
    model2.load_state_dict(torch.load("../input/saint-plus-model/saint_plus_model_20210103.pt_v2"))
except:
    model2.load_state_dict(torch.load("../input/saint-plus-model/saint_plus_model_20210103.pt_v2", map_location='cpu'))
model2.to(device)
model2.eval()

SAINTModel(
  (e_embedding): Embedding(13524, 256)
  (c_embedding): Embedding(8, 256)
  (pos_embedding): Embedding(99, 256)
  (res_embedding): Embedding(3, 256)
  (elapsed_time_embedding): Embedding(301, 256)
  (lag_embedding1): Embedding(301, 256)
  (lag_embedding2): Embedding(1441, 256)
  (lag_embedding3): Embedding(366, 256)
  (exp_embedding): Embedding(3, 256)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dro

In [11]:
class TestDataset(Dataset):
    def __init__(self, samples, test_df, n_skills, max_seq=MAX_SEQ): 
        super(TestDataset, self).__init__()
        self.samples = samples
        self.user_ids = [x for x in test_df["user_id"].unique()]
        self.test_df = test_df
        self.n_skill = n_skills
        self.max_seq = max_seq

    def __len__(self):
        return self.test_df.shape[0]

    def __getitem__(self, index):
        test_info = self.test_df.iloc[index]

        user_id = test_info["user_id"]
        target_id = test_info["content_id"]
        part = test_info["part"]
        pri_quest_elap = test_info["prior_question_elapsed_time"]
        time_lag = test_info["time_lag"]
        pri_quest_exp = test_info["prior_question_had_explanation"]
        
        q = np.zeros(self.max_seq, dtype=int)
        qa = np.zeros(self.max_seq, dtype=int)
        res = np.zeros(self.max_seq, dtype=int)
        p = np.zeros(self.max_seq, dtype=int)
        pri_elap = np.zeros(self.max_seq, dtype=int)
        lag = np.zeros(self.max_seq, dtype=int)
        pri_exp = np.zeros(self.max_seq, dtype=int)

        if user_id in self.samples.index:
            q_, qa_, p_, pri_elap_, lag_, pri_exp_ = self.samples[user_id]
            
            seq_len = len(q_)
            
            ## for zero padding
            q_ = q_+1
            pri_exp_ = pri_exp_ + 1
            res_ = qa_ + 1
            

            if seq_len >= self.max_seq:
                q = q_[-self.max_seq:]
                qa = qa_[-self.max_seq:]
                res = res_[-self.max_seq:]
                p = p_[-self.max_seq:]
                pri_elap = pri_elap_[-self.max_seq:]
                lag = lag_[-self.max_seq:]
                pri_exp = pri_exp_[-self.max_seq:]
                
            else:
                q[-seq_len:] = q_
                qa[-seq_len:] = qa_
                res[-seq_len:] = res_
                p[-seq_len:] = p_
                pri_elap[-seq_len:] = pri_elap_
                lag[-seq_len:] = lag_
                pri_exp[-seq_len:] = pri_exp_
                
        

        exercise = np.append(q[2:], [target_id+1])
        part = np.append(p[2:], [part])
        elap = np.append(pri_elap[2:], [pri_quest_elap])
        lag = np.append(lag[2:], [time_lag])
        pri_exp = np.append(pri_exp[2:], [pri_quest_exp+1])

        response = res[1:]

        return  exercise, part, response, elap, lag, pri_exp

# Catboost Feature Pipeline

# new_feature 0104 

In [12]:
lt_correct_dict = pickle.load(open('../input/arvis-feature/last_timestamp_correct.pkl', 'rb'))
np_uq_td =  pickle.load(open("../input/uq-data/np_uq_td_0518.pkl.data","rb")) 
curr_u_dict = pickle.load(open("../input/uq-data/curr_u_dict_0614_only_user_three_time_diff.pkl.data","rb"))
max_timestamp_u_dict = pickle.load(open("../input/arvis-feature/max_timestamp_u_dict_2015.pkl","rb")) 
max_timestamp_u_dict2 = pickle.load(open("../input/arvis-feature/max_timestamp_u_dict2_2015.pkl","rb")) 
max_timestamp_u_dict3 = pickle.load(open("../input/arvis-feature/max_timestamp_u_dict3_2015.pkl","rb")) 

In [13]:
!cp ../input/uq-data/user_ques_db.db ./user_ques_db.db

In [14]:
def add_uq_feats_and_update(df):
    conn = sqlite3.connect('user_ques_db.db')
    cursor = conn.cursor()
    global idx
    uq_timediff = np.zeros(len(df), dtype=np.uint64) 
    for cnt,row in enumerate(df[['user_id','content_id','timestamp']].itertuples(index=False)): 
        cursor.execute(f'select idx from user where user_id = {row[0]} and content_id = {row[1]}')
        tmp_idx = cursor.fetchall()
        if tmp_idx == []: # no got the idx for user_id-content_id pair
            uq_timediff[cnt] = 0
            np_uq_td[idx] = row[2]
            cursor.execute(f'insert into user (user_id, content_id, idx) values ({row[0]}, {row[1]}, {idx})')
            idx += 1
        else: # got the idx for user_id-content_id pair
            tmp_idx = tmp_idx[0][0]
            uq_timediff[cnt] = row[2] - np_uq_td[tmp_idx]
            np_uq_td[tmp_idx] = row[2]

    cursor.close()
    conn.commit()
    conn.close()
    
    uq_feats_df = pd.DataFrame({'curr_uq_time_diff':uq_timediff}) 
    df = pd.concat([df, uq_feats_df], axis=1)
    return df

In [15]:
def add_user_feats_without_update(df):
    utdiff = np.zeros(len(df), dtype=np.uint64)
    utdiff_mean = np.zeros(len(df), dtype=np.uint64) 
    uelapdiff = np.zeros(len(df), dtype=np.float32)  
    for cnt,row in enumerate(df[['user_id','timestamp','prior_question_elapsed_time']].itertuples(index=False)): 
        if row[0] in curr_u_dict:
            utdiff[cnt] = row[1] - curr_u_dict[row[0]]["uts"]
            utdiff_mean[cnt] = curr_u_dict[row[0]]["utsdiff"][1] / curr_u_dict[row[0]]["utsdiff"][0]
            uelapdiff[cnt] = row[2] - curr_u_dict[row[0]]["uelapdiff"]
        else:
            utdiff[cnt] = 0; utdiff_mean[cnt] = 0; uelapdiff[cnt] = 0;
            
    user_feats_df = pd.DataFrame({'curr_user_time_diff':utdiff, 'curr_user_time_diff_mean':utdiff_mean, 
                                  'curr_user_elapsed_time_diff':uelapdiff
                                 }) 
    user_feats_df['curr_user_elapsed_time_diff'].fillna(0, inplace=True) 
    df = pd.concat([df, user_feats_df], axis=1)
    return df

def update_user_feats(df):
    for cnt,row in enumerate(df[['user_id','content_id','answered_correctly','timestamp','prior_question_elapsed_time']].itertuples(index=False)): 
        if row[0] in curr_u_dict:
            curr_u_dict[row[0]]["uts"] = row[3]
            curr_u_dict[row[0]]["utsdiff"][0] += 1 
            curr_u_dict[row[0]]["utsdiff"][1] += row[3] 
            curr_u_dict[row[0]]["uelapdiff"] = row[4] 
        else:
            curr_u_dict[row[0]] = {}
            curr_u_dict[row[0]]["uts"] = row[3]
            curr_u_dict[row[0]]["utsdiff"] = [1, row[3]] 
            curr_u_dict[row[0]]["uelapdiff"] = row[4] 

In [16]:
## only in training！！
def add_user_feats(df):
    utdiff = np.zeros(len(df), dtype=np.uint64)
    utdiff_mean = np.zeros(len(df), dtype=np.uint64) 
    uelapdiff = np.zeros(len(df), dtype=np.float32)  
    for cnt,row in enumerate(tqdm(df[['user_id','content_id','answered_correctly',
                                      'timestamp','prior_question_elapsed_time',
                                     ]].itertuples(index=False),total=df.shape[0])): 
        if row[0] in curr_u_dict:
            # 写入np
            utdiff[cnt] = row[3] - curr_u_dict[row[0]]["uts"]
            utdiff_mean[cnt] = curr_u_dict[row[0]]["utsdiff"][1] / curr_u_dict[row[0]]["utsdiff"][0]
            uelapdiff[cnt] = row[4] - curr_u_dict[row[0]]["uelapdiff"]
            # 写入字典
            curr_u_dict[row[0]]["uts"] = row[3]
            curr_u_dict[row[0]]["utsdiff"][0] += 1 
            curr_u_dict[row[0]]["utsdiff"][1] += row[3] 
            curr_u_dict[row[0]]["uelapdiff"] = row[4] 
        else:
            # 写入np
            utdiff[cnt] = 0; utdiff_mean[cnt] = 0; uelapdiff[cnt] = 0;
            # 写入字典
            curr_u_dict[row[0]] = {}
            curr_u_dict[row[0]]["uts"] = row[3]
            curr_u_dict[row[0]]["utsdiff"] = [1, row[3]] 
            curr_u_dict[row[0]]["uelapdiff"] = row[4] 
            
    user_feats_df = pd.DataFrame({
                                  'curr_user_time_diff':utdiff, 'curr_user_time_diff_mean':utdiff_mean, 
                                  'curr_user_elapsed_time_diff':uelapdiff
                                 }) 

    user_feats_df['curr_user_elapsed_time_diff'].fillna(0, inplace=True) 
    df = pd.concat([df, user_feats_df], axis=1)
    return df
# curr_u_dict = {}
# train_df = add_user_feats(train_df)
# valid_df = add_user_feats(valid_df)

In [17]:
def lagtime_for_test(df):
    lagtime_mean = 0
    lagtime_mean2 = 0
    lagtime_mean3 = 0
    lagtime = np.zeros(len(df), dtype=np.float32)
    lagtime2 = np.zeros(len(df), dtype=np.float32)
    lagtime3 = np.zeros(len(df), dtype=np.float32)
    for i, (user_id,
            content_type_id,
            timestamp,
            content_id,) in enumerate(zip(df['user_id'].values, df['content_type_id'].values, df['timestamp'].values, df['content_id'].values)):
        if content_type_id==0:
            if user_id in max_timestamp_u_dict['max_time_stamp'].keys():
                lagtime[i]=timestamp-max_timestamp_u_dict['max_time_stamp'][user_id]
                if(max_timestamp_u_dict2['max_time_stamp2'][user_id]==lagtime_mean2):
                    lagtime2[i]=lagtime_mean2
                    lagtime3[i]=lagtime_mean3
                else:
                    lagtime2[i]=timestamp-max_timestamp_u_dict2['max_time_stamp2'][user_id]
                    if(max_timestamp_u_dict3['max_time_stamp3'][user_id]==lagtime_mean3):
                        lagtime3[i]=lagtime_mean3
                    else:
                        lagtime3[i]=timestamp-max_timestamp_u_dict3['max_time_stamp3'][user_id]
                    max_timestamp_u_dict3['max_time_stamp3'][user_id]=max_timestamp_u_dict2['max_time_stamp2'][user_id]
                max_timestamp_u_dict2['max_time_stamp2'][user_id]=max_timestamp_u_dict['max_time_stamp'][user_id]
                max_timestamp_u_dict['max_time_stamp'][user_id]=timestamp
            else:
                lagtime[i]=lagtime_mean
                max_timestamp_u_dict['max_time_stamp'].update({user_id:timestamp})
                lagtime2[i]=lagtime_mean2
                max_timestamp_u_dict2['max_time_stamp2'].update({user_id:lagtime_mean2})
                lagtime3[i]=lagtime_mean3
                max_timestamp_u_dict3['max_time_stamp3'].update({user_id:lagtime_mean3})
    df["lag_time"]= lagtime
    df["lag_time2"]= lagtime2
    df["lag_time3"]= lagtime3
    df["lag_time"].fillna(-1, inplace=True)
    df["lag_time2"].fillna(-1, inplace=True)
    df["lag_time3"].fillna(-1, inplace=True)
    df['lag_time'] = df['lag_time'].replace(0, method="ffill")
    df['lag_time2'] = df['lag_time2'].replace(0, method="ffill")
    df['lag_time3'] = df['lag_time3'].replace(0, method="ffill")
    
    df["lag_time"] = df["lag_time"].astype("uint64")
    df["lag_time2"] = df["lag_time2"].astype("uint64")
    df["lag_time3"] = df["lag_time3"].astype("uint64")
    return df

In [18]:
# функция расчета кумулятивной суммы по переменной и обновления словаря
# cumulative sum and dict update
def add_feats(df_np, feat_dict, col_idx, col_feat):
    '''
    df_np - pandas датафрейм приведенный к numpy .to_numpy()
    feat_dict - numpy словарь вида [idx - feat]
    col_idx - индекс столбца переменной по которой будем агрегировать
    col_feat - индекс столбца переменной по которой будем считать кумулятивную сумму
    '''       
    current_feat_value = np.zeros(len(df_np))
    for cnt, row in enumerate(df_np[:,[col_idx, col_feat]]):
        current_feat_value[cnt] = feat_dict[row[0]] # текущее значение из словаря
        feat_dict[row[0]] += row[1] # в словарь добавляем текущее значение строки
    # заменяем обновленными значениями
    df_np[:, col_feat] = current_feat_value
    
    return df_np


# функция добавления данных из словаря (новый пользователь = добавляем с 0)
def add_feats_from_dict(df_np, feat_dict, col_idx, col_dict=-1):
    '''
    df_np - pandas датафрейм приведенный к numpy .to_numpy()
    feat_dict - numpy словарь вида [idx - feat]
    col_idx - индекс столбца переменной по которой добавлять данные из словаря
    '''       
    current_feat_value = np.zeros(len(df_np))
    for cnt, idx in enumerate(df_np[:,col_idx]):
        if col_dict == -1: current_feat_value[cnt] = feat_dict[idx] # текущее значение из словаря = значение
        else: current_feat_value[cnt] = feat_dict[idx][col_dict] # текущее значение из словаря = значение из списка с индексом col_dict
       
    return (np.c_[ df_np, current_feat_value ])
    #return np.concatenate((df_np, current_feat_value), axis=1)

# функция добавления данных из словаря (новый индекс = забираем данные с индексом = -100)
def add_feats_from_dict_got_new_user(df_np, feat_dict, col_idx, col_dict=-1):
    '''
    df_np - pandas датафрейм приведенный к numpy .to_numpy()
    feat_dict - numpy словарь вида [idx - feat]
    col_idx - индекс столбца переменной по которой добавлять данные из словаря
    '''       
    current_feat_value = np.zeros(len(df_np))
    for cnt, idx in enumerate(df_np[:,col_idx]):
        # row[0] = index
        # row[1] = value
        if idx in feat_dict.keys(): # если индекс есть в словаре - берем данные по его id   
            if col_dict == -1: current_feat_value[cnt] = feat_dict[idx] # текущее значение из словаря = значение
            else: current_feat_value[cnt] = feat_dict[idx][col_dict] # текущее значение из словаря = значение из списка с индексом col_dict
        else: # если индекса нет в словаре - берем данные по id = -100
            if col_dict == -1: current_feat_value[cnt] = feat_dict[-100] # текущее значение из словаря = значение
            else: current_feat_value[cnt] = feat_dict[-100][col_dict] # текущее значение из словаря = значение из списка с индексом col_dict
            feat_dict[idx] = feat_dict[-100] # добавляем новый индекс
       
    return (np.c_[ df_np, current_feat_value ])
    #return np.concatenate((df_np, current_feat_value), axis=1)


# функция обновления словаря
def update_dict(df_pd, feat_dict, col_idx, col_feat, col_dict=-1):
    for row in df_pd[['content_type_id', col_idx, col_feat]].values:
        if row[0] == 0:
            if col_dict == -1: feat_dict[row[1]] += row[2] # добавляем текущее значение
            else: feat_dict[row[1]][col_dict] += row[2] # добавляем текущее значение в стобец
            
# добавление ohe во фрейм для заданной переменной и ее значения
def add_ohe(df, col_feat, oh_value):
    # df - общий фрейм np array
    # col_feat - индекс переменной
    return (np.c_[ df, np.array([int(i == oh_value) for i in df[:,col_feat]]) ])  
    #return np.concatenate((df, np.array([int(i == oh_value) for i in df[:,col_feat]])), axis=1)


# user_slice_accuracy_n - точность ответа на последних N вопросах
# функция добавления данных из словаря
def user_slice_accuracy_n_get(df_pd, feat_dict):
    global_first_question_accuracy = 0.6453965159034877     
    current_list = np.zeros(len(df_pd))
    for cnt, (user, content_type_id) in enumerate(df_pd[['user_id', 'content_type_id']].values):  
        # если вопрос
        if content_type_id == 0:
            # если user_id есть в словаре
            if user in feat_dict:   
                current_list[cnt] = np.mean(feat_dict[user])
            else: # если user_id нет в словаре - берем global_first_question_accuracy
                current_list[cnt] = 0.6454
        else:
            current_list[cnt] = 0
    return current_list
# user_slice_accuracy_n - точность ответа на последних N вопросах   
# функция добавления обновления словаря
def user_slice_accuracy_n_update(df_pd, feat_dict, border=5):   
    for cnt, (user, answer) in enumerate(df_pd[['user_id', 'answered_correctly']].values):  
        # если user_id есть в словаре
        if user in feat_dict:   
            feat_dict[user].append(answer) # добавляем текущий ответ
            feat_dict[user] = feat_dict[user][-border:] # ограничиваем количество
        # если user_id нет в словаре
        else: 
            feat_dict[user] = [answer] # добавляем текущий ответ         
    return feat_dict

# user_slice_accuracy_session - точность ответа на последней сессии
# функция добавления данных из словаря
def user_slice_accuracy_session_get(df_pd, feat_dict, session_max_time=12):  
    current_list = np.zeros(len(df_pd))
    for cnt, (user, timestamp, content_type_id) in enumerate(df_pd[['user_id', 'timestamp', 'content_type_id']].values):  
        # если вопрос
        if content_type_id == 0:
            # если user_id есть в словаре
            if user in feat_dict:  
                # считаем дельту timestamp в часах
                time_delta_h = (timestamp - feat_dict[user][1]) / 1000 / 60 / 60
                if time_delta_h < session_max_time: 
                    current_list[cnt] = np.mean(feat_dict[user][0])
                else: 
                    current_list[cnt] = 0.67
            # если user_id нет в словаре
            else: 
                current_list[cnt] = 0.67  
        # если лекция
        else: current_list[cnt] = 0
    return current_list
# user_slice_accuracy_session - точность ответа на последней сессии
# функция обновления словаря
def user_slice_accuracy_session_update(df_pd, feat_dict, session_max_time=12):  
    for cnt, (user, answer, timestamp) in enumerate(df_pd[['user_id', 'answered_correctly', 'timestamp']].values):  
        # если user_id есть в словаре
        if user in feat_dict:  
            # считаем дельту timestamp в часах
            time_delta_h = (timestamp - feat_dict[user][1]) / 1000 / 60 / 60
            if time_delta_h < session_max_time: 
                feat_dict[user][0].append(answer)
                feat_dict[user][1] = timestamp
            else: 
                feat_dict[user][0] = [answer]
                feat_dict[user][1] = timestamp
        # если user_id нет в словаре
        else: 
            feat_dict[user] = [[answer], timestamp]
    return feat_dict
   

# количество вопросов по content_id на которые отвечал каждый user_id накопленно: берем данные + обновляем словарь    
def user_question_attempt_cnt_get_update(df_pd, feat_dict):
    current_feat_value = np.zeros(len(df_pd))
    for idx, (user_id, content_id, content_type_id) in enumerate(df_pd[['user_id', 'content_id', 'content_type_id']].values):
        
        # если вопрос
        if content_type_id == 0:   
            current_feat_value[idx] = list(feat_dict[user_id]).count(content_id)
            # обновляем = добавляем content_id в лист
            feat_dict[user_id] = np.append(feat_dict[user_id], content_id)
        # если лекция - ставим 0
        else:
            current_feat_value[idx] = 0

    return current_feat_value

# part_l_q_cnt - функция добавления данных из словаря + обновление словаря
def user_lectures_part(df_pd, feat_dict):      
    current_list = np.zeros(len(df_pd))
    for cnt, (user, content_type_id, part_q, part_l) in enumerate(df_pd[['user_id', 'content_type_id', 'part_q', 'part_l']].values):
        part_q = max(0, part_q) # для значения -100 (новый id)
        
        # если вопрос
        if content_type_id == 0:
            # если user_id есть в словаре - берем данные по его id 
            if user in feat_dict:            
                current_list[cnt] = feat_dict[user][part_q]   
            # если user_id нет в словаре - добавляем с нулями, забираем 0
            else: 
                feat_dict[user] = [0] * 8
                current_list[cnt] = 0  

        # если лекция
        else:
            # если user_id есть в словаре - обновляем словарь
            if user in feat_dict:            
                 feat_dict[user][part_l] += 1  
            # если user_id нет в словаре - добавляем с нулями
            else: 
                feat_dict[user] = [0] * 8
                feat_dict[user][part_l] += 1 
            # забираем 0
            current_list[cnt] = 0
    
    return current_list

# lecture_cnt - функция добавления данных из словаря
def user_lecture_cnt(df_pd, feat_dict):
    '''
    df_pd - pandas датафрейм приведенный к numpy .to_numpy()
    feat_dict - словарь
    '''       
    current_list = np.zeros(len(df_pd))
    for cnt, user in enumerate(df_pd['user_id'].values):
        if user in feat_dict: # если user_id есть в словаре - берем данные по его id             
            current_list[cnt] = sum(feat_dict[user])           
        else: # если user_id нет в словаре -  забираем 0
            current_list[cnt] = 0  
    
    return current_list


# tag_l_q_equal_cnt - слушал ли юзер лекцию по тегу равную тегу вопроса (количество) - функция добавления данных из словаря + обновление словаря
def user_l_q_tag_equal(df_pd, feat_dict):
    '''
    df_pd - pandas датафрейм приведенный к numpy .to_numpy()
    feat_dict - словарь
    border - количество последних ответов к отслеживанию
    '''       
    current_list = np.zeros(len(df_pd))
    for idx, (user, content_type_id, tag, tags_list) in enumerate(df_pd[['user_id', 'content_type_id', 'tag_l', 'tags_list']].values):
        # если вопрос
        if content_type_id == 0:
            # количество тегов = тегу прослушенной лекции
            current_list[idx] = len(set(feat_dict[user]) & set(tags_list))
        # если лекция
        else:
            feat_dict[user].append(int(tag)) # добавляем текущий тег лекции
            current_list[idx] = 0
    
    return current_list

# данные из словарей вопросов и лекций
def get_q_l(df_pd, feat_dict, key):
    current_list = []
    for idx, content_id in enumerate(df_pd['content_id'].values):
        if content_id in feat_dict:
            current_list.append(feat_dict[content_id][key])
        else:
            current_list.append(-100)
    return current_list

# функция обновления словаря - возвращает датафрейм с новым значением
def dict_user_timestampsdelta_get_update_3(df, feat_dict):
    #if need_sort: df = df.sort_values(['user_id', 'timestamp'])
    q_list = np.zeros((len(df), 3), dtype = np.float32)
    l_list = np.zeros((len(df), 3), dtype = np.float32)
    for cnt, (user_id, timestamp, content_type_id) in enumerate(df[['user_id', 'timestamp', 'content_type_id']].values):
        # переводим в минуты
        timestamp = timestamp / 1000 / 60
        # если user_id есть в словаре - берем значения, обновляем словарь
        if user_id in feat_dict:
            # вычитаем из текущего времени каждый элемент из словаря
            q_list[cnt] = np.array([timestamp - t for t in feat_dict[user_id][0]]) # вопросы
            l_list[cnt] = np.array([timestamp - t for t in feat_dict[user_id][1]]) # лекции
            # обновляем значения в словаре (content_type_id==0 вопрос, content_type_id==1 лекция)
            feat_dict[user_id][int(content_type_id)].pop(0) # удаляем первый элемент
            feat_dict[user_id][int(content_type_id)].append(timestamp) # добавляем в конец текущее время
                
        # если user_id нет - добавляем и забираем с np.nan
        else:
            if content_type_id == 1: feat_dict[user_id] = [[np.nan, np.nan, np.nan], [np.nan, np.nan, 0]]
            else: feat_dict[user_id] = [[np.nan, np.nan, 0], [np.nan, np.nan, np.nan]]
            q_list[cnt] = np.array([np.nan, np.nan, np.nan])
            l_list[cnt] = np.array([np.nan, np.nan, np.nan])
            
    # добавляем в основной датафрейм
    for i in [0, 1, 2]:
        df['prior_question_' + str(i+1) + '_timedelta_min'] = q_list[:, i]
        df['prior_lecture_' + str(i+1) + '_timedelta_min'] = l_list[:, i]
        df['prior_question_' + str(i+1) + '_timedelta_min'] = df['prior_question_' + str(i+1) + '_timedelta_min'].fillna(-100).replace(0, method='ffill')
        df['prior_lecture_' + str(i+1) + '_timedelta_min'] = df['prior_lecture_' + str(i+1) + '_timedelta_min'].fillna(-100).replace(0, method='ffill')
        
    del [q_list, l_list]
    return df

# {user: [[теги 0-N], [количество вопросав по тегу], [количество правильных ответов по тегу]]}
# для работы нужен подгруженный словарь dict_global_question_tag_accuracy
def user_question_tag_accuracy_get(df, feat_calc):
    # задаем веса для тегов
    tags_w = [0.43, 0.27, 0.18, 0.08, 0.03, 0.01]   
    values = np.zeros(len(df))
    feat_list = ['user_id', 'content_type_id', 'tags_list']
    for cnt, (user, content_type_id, tags_list) in enumerate(df[feat_list].values):
        if tags_list == -100: tags_list = [-100]
        # если вопрос:
        if content_type_id == 0:
            # если юзер в словаре
            if user in feat_calc:
                user_tags_accuracy = 0
                # для каждого тега
                for tag_i in tags_list:
                    tags_accuracy_list = []
                    # если тег есть в словаре - забирем текущие значения
                    if tag_i in feat_calc[user][0]:
                        # находим индекс тега
                        tag_i_idx = feat_calc[user][0].index(tag_i)
                        # добавляем точность текущего тега по юзеру
                        tags_accuracy_list.append(feat_calc[user][2][tag_i_idx] / feat_calc[user][1][tag_i_idx])
                    # если тега нет в словаре
                    else:
                        # добавляем среднюю точность текущего тега из словаря
                        tags_accuracy_list.append(dict_global_question_tag_accuracy[tag_i][2])
                # считаем общую точность по текущим тегам для юзеру
                l = len(tags_accuracy_list)
                tags_w_l = tags_w[:l]
                tags_w_l_sum = sum(tags_w_l)
                tags_w_current = [x/tags_w_l_sum for x in tags_w_l]                    
                user_tags_accuracy = sum([tag * w for tag, w in zip(tags_accuracy_list, tags_w_current)])

            # если юзера нет в словаре - добавляем
            else:
                # для каждого тега
                for tag_i in tags_list:
                    tags_accuracy_list = []
                    # добавляем среднюю точность текущего тега из словаря
                    tags_accuracy_list.append(dict_global_question_tag_accuracy[tag_i][2])
                # считаем общую точность по текущим тегам для юзера
                l = len(tags_accuracy_list)
                tags_w_l = tags_w[:l]
                tags_w_l_sum = sum(tags_w_l)
                tags_w_current = [x/tags_w_l_sum for x in tags_w_l]                    
                user_tags_accuracy = sum([tag * w for tag, w in zip(tags_accuracy_list, tags_w_current)])

            values[cnt] = user_tags_accuracy
        # если лекция:
        else:
            values[cnt] = 0
        
    return values

def user_question_tag_accuracy_update(df, feat_calc):
    feat_list = ['user_id', 'answered_correctly', 'content_type_id', 'tags_list']
    for cnt, (user, answer, content_type_id, tags_list) in enumerate(df[feat_list].values):
        if tags_list == -100: tags_list = [-100]
        # если вопрос:
        if content_type_id == 0:
            # если юзер в словаре
            if user in feat_calc:
                user_tags_accuracy = 0
                # для каждого тега
                for tag_i in tags_list:
                    tags_accuracy_list = []
                    # если тег есть в словаре
                    if tag_i in feat_calc[user][0]:
                        # находим индекс тега
                        tag_i_idx = feat_calc[user][0].index(tag_i)
                        # обновляем словарь
                        feat_calc[user][1][tag_i_idx] += 1
                        feat_calc[user][2][tag_i_idx] += answer
                    # если тега нет в словаре
                    else:
                        # добавляем тег в словарь
                        feat_calc[user][0].append(tag_i)
                        feat_calc[user][1].append(1)
                        feat_calc[user][2].append(answer)

            # если юзера нет в словаре - добавляем
            else:
                feat_calc[user] = [[], [], []]
                # для каждого тега
                for tag_i in tags_list:
                    feat_calc[user][0].append(tag_i)
                    feat_calc[user][1].append(1)
                    feat_calc[user][2].append(answer)
        
    return feat_calc


def user_correct_incorrect_timestamp_get(df, feat_dict):
    '''
    Время до последнего правильного и неправильного ответа
    Функция возвращает лист с данными
    '''
    incorrect_list = []
    correct_list = []
    for (user_id, timestamp, content_type_id) in df[['user_id', 'timestamp', 'content_type_id']].values:
        # переводим в минуты
        timestamp = timestamp / 1000 / 60
        correct_value, incorrect_value = 0, 0
        # если лекция
        if content_type_id: 
            incorrect_list.append(-100)
            correct_list.append(-100)
        # если вопрос
        else:
            # если user_id есть в словаре - берем значения
            if user_id in feat_dict:
                # забираем текущие значения timestamp из словаря
                incorrect_value, correct_value = feat_dict[user_id][0], feat_dict[user_id][1]
            # если user_id нет
            else: 
                # забираем np.nan
                incorrect_value, correct_value = np.nan, np.nan

            # считаем дельту 
            incorrect_value = (timestamp - incorrect_value)
            correct_value = (timestamp - correct_value)
            # добавляем    
            incorrect_list.append(incorrect_value)
            correct_list.append(correct_value)

    # добавляем в датафрейм
    df['prior_question_incorrect_timedelta_min'] = incorrect_list
    df['prior_question_correct_timedelta_min'] = correct_list
    # заполянем пропуски, заменяем 0 на предыдущее значение отличное от 0
    df['prior_question_incorrect_timedelta_min'] = df['prior_question_incorrect_timedelta_min'].fillna(-100).replace(0, method='ffill')
    df['prior_question_correct_timedelta_min'] = df['prior_question_correct_timedelta_min'].fillna(-100).replace(0, method='ffill')
   
    return df

def user_correct_incorrect_timestamp_update(df, feat_dict):
    '''
    Время до последнего правильного и неправильного ответа
    Функция возвращает лист с данными и обновляет словарь
    '''
    incorrect_list = []
    correct_list = []
    for (user_id, timestamp, content_type_id, answer) in df[['user_id', 'timestamp', 'content_type_id', 'answered_correctly']].values:
        # переводим в минуты
        timestamp = timestamp / 1000 / 60
        # если вопрос
        if content_type_id == 0: 
            # если user_id есть в словаре - обновляем словарь
            if user_id in feat_dict:
                if answer: feat_dict[user_id][1] = timestamp  # если ответ верный
                else: feat_dict[user_id][0] = timestamp # если ответ не верный
            # если user_id нет
            else: 
                # добавляем юзера в словарь
                if answer: feat_dict[user_id] = [np.nan, timestamp]  # если ответ верный
                else: feat_dict[user_id] = [timestamp, np.nan] # если ответ не верный

    return feat_dict


time_session_map = {'q_count_all' : 0,
                    'q_count_n' : 1,
                    'time_all_n' : 2,
                    'time_n' : 3,
                    'time_dict_all' : 4,
                    'time_dict_n' : 5,
                    'prior_timestamp' : 6,
                    'prior_container' : 7,
                    'prior_container_shape' : 8,
                    'prior_content_id' : 9,
                   }
def user_slice_question_time_mean_session(df_pd, feat_dict, session_max_time_min = 180): 
    prior_question_elapsed_time_mean = 25452.541
    out_mean_n = np.zeros(len(df_pd))
    out_mean_all = np.zeros(len(df_pd))
    out_delta_n = np.zeros(len(df_pd))
    out_delta_all = np.zeros(len(df_pd))
    
    calc_list = ['user_id', 'timestamp', 'task_container_id', 'content_id', 
                 'content_type_id', 'prior_question_elapsed_time', 'task_container_freq']
    for cnt, (user, timestamp, task_container_id, content_id, content_type_id, prior_question_elapsed_time, task_container_freq) in enumerate(df_pd[calc_list].values):
        # время в минуты
        timestamp = timestamp / 1000 / 60
        if content_id not in dict_content_elapsed_time_mean: content_id = -100
        # если вопрос
        if content_type_id == 0:
            # если user_id есть в словаре
            if user in feat_dict:  
                # считаем дельту timestamp
                time_delta = (timestamp - feat_dict[user][time_session_map['prior_timestamp']])
                if time_delta < session_max_time_min: 
                    # если новый контейнер
                    if task_container_id != feat_dict[user][time_session_map['prior_container']]:
                        s = feat_dict[user][time_session_map['prior_container_shape']]
                        c = feat_dict[user][time_session_map['prior_content_id']]
                        # обновляем словарь
                        feat_dict[user][time_session_map['time_all_n']] += prior_question_elapsed_time * s
                        feat_dict[user][time_session_map['time_n']] += prior_question_elapsed_time * s
                        feat_dict[user][time_session_map['q_count_all']] += s
                        feat_dict[user][time_session_map['q_count_n']] += s
                        feat_dict[user][time_session_map['time_dict_all']] += dict_content_elapsed_time_mean[c] * s
                        feat_dict[user][time_session_map['time_dict_n']] += dict_content_elapsed_time_mean[c] * s
                        feat_dict[user][time_session_map['prior_timestamp']] = timestamp
                        feat_dict[user][time_session_map['prior_container']] = task_container_id  
                        feat_dict[user][time_session_map['prior_content_id']] = content_id 
                        feat_dict[user][time_session_map['prior_container_shape']] = task_container_freq
                        # считаем фичи                          
                        out_mean_n[cnt] = feat_dict[user][time_session_map['time_n']] / feat_dict[user][time_session_map['q_count_n']]
                        out_mean_all[cnt] = feat_dict[user][time_session_map['time_all_n']] / feat_dict[user][time_session_map['q_count_all']]
                        out_delta_n[cnt] = feat_dict[user][time_session_map['time_n']] / feat_dict[user][time_session_map['time_dict_n']]
                        out_delta_all[cnt] = feat_dict[user][time_session_map['time_all_n']] / feat_dict[user][time_session_map['time_dict_all']]

                    # если контейнер прежний - берем предыдущее значение
                    else:
                        out_mean_n[cnt] = out_mean_n[cnt-1]
                        out_mean_all[cnt] = out_mean_all[cnt-1]
                        out_delta_n[cnt] = out_delta_n[cnt-1]
                        out_delta_all[cnt] = out_delta_all[cnt-1]
                else: 
                    # если новый контейнер
                    if task_container_id != feat_dict[user][time_session_map['prior_container']]:
                        s = feat_dict[user][time_session_map['prior_container_shape']]
                        c = feat_dict[user][time_session_map['prior_content_id']]
                        # обновляем словарь
                        feat_dict[user][time_session_map['time_all_n']] += prior_question_elapsed_time * s
                        feat_dict[user][time_session_map['time_n']] = 0
                        feat_dict[user][time_session_map['q_count_all']] += s
                        feat_dict[user][time_session_map['q_count_n']] = 0
                        feat_dict[user][time_session_map['time_dict_all']] += dict_content_elapsed_time_mean[c] * s
                        feat_dict[user][time_session_map['time_dict_n']] = 0
                        feat_dict[user][time_session_map['prior_timestamp']] = timestamp
                        feat_dict[user][time_session_map['prior_container']] = task_container_id
                        feat_dict[user][time_session_map['prior_content_id']] = content_id 
                        feat_dict[user][time_session_map['prior_container_shape']] = task_container_freq
                        # считаем фичи                            
                        out_mean_n[cnt] = feat_dict[user][time_session_map['time_all_n']] / feat_dict[user][time_session_map['q_count_all']]
                        out_mean_all[cnt] = feat_dict[user][time_session_map['time_all_n']] / feat_dict[user][time_session_map['q_count_all']]
                        out_delta_n[cnt] = feat_dict[user][time_session_map['time_all_n']] / feat_dict[user][time_session_map['time_dict_all']]
                        out_delta_all[cnt] = feat_dict[user][time_session_map['time_all_n']] / feat_dict[user][time_session_map['time_dict_all']]

                    # если контейнер прежний - берем предыдущее значение
                    else:
                        out_mean_n[cnt] = out_mean_n[cnt-1]
                        out_mean_all[cnt] = out_mean_all[cnt-1]
                        out_delta_n[cnt] = out_delta_n[cnt-1]
                        out_delta_all[cnt] = out_delta_all[cnt-1]
            # если user_id нет в словаре
            else:
                out_mean_n[cnt] = prior_question_elapsed_time_mean
                out_mean_all[cnt] = prior_question_elapsed_time_mean
                out_delta_n[cnt] = prior_question_elapsed_time_mean / dict_content_elapsed_time_mean[content_id] 
                out_delta_all[cnt] = prior_question_elapsed_time_mean / dict_content_elapsed_time_mean[content_id] 
                feat_dict[user] = [1, 1, 0, 0, 
                                   dict_content_elapsed_time_mean[content_id], dict_content_elapsed_time_mean[content_id], 
                                   timestamp, task_container_id, task_container_freq, content_id]
        # если лекция
        else: out_mean_n[cnt], out_mean_all[cnt], out_delta_n[cnt], out_delta_all[cnt] = 0, 0, 0, 0
        
    df_pd['user_question_time_mean_n_session'] = out_mean_n
    df_pd['user_question_time_mean_all_session'] = out_mean_all
    df_pd['user_question_time_delta_n_session'] = out_delta_n
    df_pd['user_question_time_delta_all_session'] = out_delta_all
    return df_pd


# prior_question_had_explanation counter features
user_priorq_expl_types_map = {'q_count' : 0,
                              'c_t_cnt' : 1,
                              'c_f_cnt' : 2,
                              'w_t_cnt' : 3,
                              'w_f_cnt' : 4,
                              'prior_container' : 5,
                              'prior_container_shape' : 6,
                              'prior_answer_expl_type' : 7,
                             }

def user_priorq_expl_types_get(df_pd, feat_dict): 
    c_t_cnt = np.zeros(len(df_pd))
    c_f_cnt = np.zeros(len(df_pd))
    w_t_cnt = np.zeros(len(df_pd))
    w_f_cnt = np.zeros(len(df_pd))
    cw_tf_type = np.zeros(len(df_pd))
    
    calc_list = ['user_id', 'task_container_id', 'content_type_id', 
                 'prior_question_had_explanation', 'task_container_freq']
    for cnt, (user, task_container_id, content_type_id, priorq_had_expl, task_container_freq) in enumerate(df_pd[calc_list].values):
        # если вопрос
        if content_type_id == 0:
            # если user_id есть в словаре
            if user in feat_dict:
                # если контейнер новый
                if task_container_id != feat_dict[user][user_priorq_expl_types_map['prior_container']]:
                    # количество вопросов в предыдущем контейнере 
                    prior_c_shape = feat_dict[user][user_priorq_expl_types_map['prior_container_shape']]
                    # общее количество вопросов
                    q_count = feat_dict[user][user_priorq_expl_types_map['q_count']] + 0.0001
                    # 1.1. возвращаем: c_t_cnt/cnt, c_f_cnt/cnt, w_t_cnt/cnt, w_f_cnt/cnt, cw_tf_type
                    c_t_cnt[cnt] = feat_dict[user][user_priorq_expl_types_map['c_t_cnt']] / q_count
                    c_f_cnt[cnt] = feat_dict[user][user_priorq_expl_types_map['c_f_cnt']] / q_count
                    w_t_cnt[cnt] = feat_dict[user][user_priorq_expl_types_map['w_t_cnt']] / q_count
                    w_f_cnt[cnt] = feat_dict[user][user_priorq_expl_types_map['w_f_cnt']] / q_count
                    cw_tf_type[cnt] = feat_dict[user][user_priorq_expl_types_map['prior_answer_expl_type']]
                # если контейнер старый
                else:
                    # 2.1. возвращаем: предыдущие значения (счетчик -1)
                    c_t_cnt[cnt] = c_t_cnt[cnt - 1]
                    c_f_cnt[cnt] = c_f_cnt[cnt - 1]
                    w_t_cnt[cnt] = w_t_cnt[cnt - 1]
                    w_f_cnt[cnt] = w_f_cnt[cnt - 1]
                    cw_tf_type[cnt] = cw_tf_type[cnt - 1]
            # если user_id нет в словаре
            else:
                c_t_cnt[cnt], c_f_cnt[cnt], w_t_cnt[cnt], w_f_cnt[cnt], w_f_cnt[cnt] = 0, 0, 0, 0, 0

        # если лекция
        else: c_t_cnt[cnt], c_f_cnt[cnt], w_t_cnt[cnt], w_f_cnt[cnt], cw_tf_type[cnt] = 0, 0, 0, 0, 0
        
    df_pd['user_prior_correct_expl_prc'] = c_t_cnt
    df_pd['user_prior_correct_noexpl_prc'] = c_f_cnt
    df_pd['user_prior_wrong_expl_prc'] = w_t_cnt
    df_pd['user_prior_wrong_noexpl_prc'] = w_f_cnt
    df_pd['user_prior_answer_expl_type'] = cw_tf_type

    return df_pd

def user_priorq_expl_types_update(df_pd, feat_dict): 
    calc_list = ['user_id', 'task_container_id', 'content_type_id', 
                 'prior_question_had_explanation', 'answered_correctly', 'task_container_freq']
    for cnt, (user, task_container_id, content_type_id, priorq_had_expl, answer, task_container_freq) in enumerate(df_pd[calc_list].values):
        if answer: t1 = 'c_'
        else: t1 = 'w_'
        if priorq_had_expl: t2 = 't_cnt'
        else: t2 = 'f_cnt'
        col = t1 + t2
        # если вопрос
        if content_type_id == 0:
            # если user_id есть в словаре
            if user in feat_dict:
                # если контейнер новый
                if task_container_id != feat_dict[user][user_priorq_expl_types_map['prior_container']]:
                    # количество вопросов в предыдущем контейнере 
                    prior_c_shape = feat_dict[user][user_priorq_expl_types_map['prior_container_shape']]
                    # 1.2. обновляем словарь: q_count += prior_c_shape
                    feat_dict[user][user_priorq_expl_types_map['q_count']] += prior_c_shape
                    # в зависимости от ответа и флага обновляем счетчик c/w_t/f_cnt += prior_c_shape                   
                    feat_dict[user][user_priorq_expl_types_map[col]] += prior_c_shape
                    # 2.3. обновляем словарь: контейнер, количество вопросов в предыдущем контейнере
                    feat_dict[user][user_priorq_expl_types_map['prior_container']] = task_container_id
                    feat_dict[user][user_priorq_expl_types_map['prior_container_shape']] = task_container_freq
                    feat_dict[user][user_priorq_expl_types_map['prior_answer_expl_type']] = user_priorq_expl_types_map[col]

            # если user_id нет в словаре
            else:
                feat_dict[user] = [0, 0, 0, 0, 0, 
                                   task_container_id, task_container_freq, 0]

    return feat_dict


# функция обновления словаря - возвращает датафрейм с новым значением
def user_lectures_typeof_cnt(df, feat_dict):
    concept = np.zeros(len(df))
    solving_question = np.zeros(len(df))
    for cnt, (user, content_id, content_type_id) in enumerate(df[['user_id', 'content_id', 'content_type_id']].values):
        # если лекция
        if content_type_id:
            # если user есть в словаре
            if user in feat_dict:
                # если content_id есть в словаре dict_lectures
                if content_id in dict_lectures:
                    # обновляем словарь
                    if dict_lectures[content_id]['type_of'] == 'concept': feat_dict[user][0] += 1
                    elif dict_lectures[content_id]['type_of'] == 'solving question': feat_dict[user][1] += 1
            # если user нет в словаре
            else:
                # если content_id есть в словаре dict_lectures
                if content_id in dict_lectures:
                    # создаем юзера
                    feat_dict[user] = [0, 0]
                    # обновляем словарь
                    if dict_lectures[content_id]['type_of'] == 'concept': feat_dict[user][0] += 1
                    elif dict_lectures[content_id]['type_of'] == 'solving question': feat_dict[user][1] += 1
            # возвращаем
            concept[cnt] = 0
            solving_question[cnt] = 0
        # если вопрос
        else:
            # если user есть в словаре
            if user in feat_dict:
                # возвращаем
                concept[cnt] = feat_dict[user][0]
                solving_question[cnt] = feat_dict[user][1]
            # если user есть в словаре
            else:
                # возвращаем
                concept[cnt] = 0
                solving_question[cnt] = 0
                
    df['lecture_concept_cnt'] = concept.astype(np.uint16)
    df['lecture_solving_question_cnt'] = solving_question.astype(np.uint16)
    return df

def user_answer_mode_n_get(df_pd, feat_dict, border=10):   
    current_list = np.zeros(len(df_pd), dtype = np.uint8)
    for cnt, (user, content_type_id, content_id) in enumerate(df_pd[['user_id', 'content_type_id', 'content_id']].values):  
        # если вопрос
        if content_type_id == 0:
            # если user_id есть в словаре
            if user in feat_dict:
                # если content_id есть в словаре dict_questions
                if content_id in dict_questions:
                    current_list[cnt] = max(set(feat_dict[user]), key=feat_dict[user].count) == dict_questions[content_id]['correct_answer']
                # если content_id нет в словаре dict_questions
                else:
                    current_list[cnt] = 100
            else: # если user_id нет в словаре
                current_list[cnt] = 100
        else:
            current_list[cnt] = 0
            
    return current_list

def user_answer_mode_n_update(df_pd, feat_dict, border=10):   
    for cnt, (user, user_answer, content_type_id, content_id) in enumerate(df_pd[['user_id', 'user_answer', 'content_type_id', 'content_id']].values):  
        # если вопрос
        if content_type_id == 0:
            # обновляем словарь
            feat_dict[user].append(user_answer) # добавляем текущий ответ
            feat_dict[user] = feat_dict[user][-border:] # ограничиваем количество
            
    return feat_dict


# создаем map по самым крупным стартовым bundle_id
first_bundle_id_map = {7900 : 1,
                       128 : 2,
                       5692 : 3,
                       -100 : 4,
                      }

# тянем bundle_id
def question_bundle_id_get(df_pd, feat_dict):   
    current_list = np.zeros(len(df_pd))
    for cnt, (content_type_id, content_id) in enumerate(df_pd[['content_type_id', 'content_id']].values):
        # если вопрос
        if content_type_id == 0: 
            # если content_id есть в словаре dict_questions
            if content_id in feat_dict:
                current_list[cnt] = feat_dict[content_id]['bundle_id']
            # если content_id нет в словаре dict_questions
            else:
                current_list[cnt] = -100
        # если лекция
        else: 
            current_list[cnt] = 0
    df_pd['bundle_id'] = current_list.astype(np.int32)
    return df_pd

# функция расчета переменной и обновления словаря {user : cluster} - кластер юзера по первому bundle_id
def user_bundle_cluster_get_update(df_pd, feat_dict):   
    l = ['user_id', 'content_type_id', 'bundle_id', 'timestamp']
    current_list = np.zeros(len(df_pd))
    for cnt, (user, content_type_id, bundle, timestamp) in enumerate(df_pd[l].values): 
        # если user нет в словаре - обновляем словарь
        if user not in feat_dict: 
            # если вопрос и он первый для юзера
            if content_type_id == 0 and timestamp == 0:   
                # если bundle есть в first_bundle_id_map
                if bundle in first_bundle_id_map:
                    # добавляем кластер
                    feat_dict[user] = first_bundle_id_map[bundle]
                # если bundle нет в first_bundle_id_map
                else: feat_dict[user] = 4  
            # иначе добавляем с 4
            else: feat_dict[user] = 4 

        # возвращаем данные
        current_list[cnt] = feat_dict[user]
        
    df_pd['first_bundle_id_cluster'] = current_list.astype(np.uint8)
    return df_pd

# функция обновления словаря - точность общая и по bundle_id для кластера
# {bundle_id : [[all_cnt, all_corr], [cl_1_cnt, cl_1_corr], [cl_2_cnt, cl_2_corr], [cl_3_cnt, cl_3_corr], [cl_4_cnt, cl_4_corr]]}
def question_bundle_accuracy_update(df_pd, feat_dict):   
    l = ['content_type_id', 'bundle_id', 'answered_correctly', 'first_bundle_id_cluster']
    for cnt, (content_type_id, bundle_id, answer, cluster) in enumerate(df_pd[l].values):
        # если вопрос
        if content_type_id == 0: 
            # если bundle_id есть в словаре
            if bundle_id in feat_dict:
                # обновляем словарь
                feat_dict[bundle_id][0][0] += 1
                feat_dict[bundle_id][0][1] += answer
                feat_dict[bundle_id][cluster][0] += 1
                feat_dict[bundle_id][cluster][1] += answer
            # если bundle_id нет в словаре - добавляем с кластером = 4
            else:
                feat_dict[bundle_id] = [[1, answer], [0, 0], [0, 0], [0, 0], [1, answer]]
                
    return feat_dict

# тянем точность по bundle_id в train из словаря dict_question_bundle_accuracy
def question_bundle_accuracy_get(df_pd, feat_dict): 
    current_list = np.zeros((len(df_pd), 2), dtype = np.float32)
    l = ['content_type_id', 'bundle_id', 'first_bundle_id_cluster']
    for cnt, (content_type_id, bundle_id, cluster) in enumerate(df_pd[l].values):
        # если вопрос
        if content_type_id == 0: 
            # если bundle_id есть в словаре
            if bundle_id in feat_dict: 
                # возвращаем значения
                current_list[cnt, 0] = feat_dict[bundle_id][0][1] / (feat_dict[bundle_id][0][0] + 0.000001)
                current_list[cnt, 1] = feat_dict[bundle_id][cluster][1] / (feat_dict[bundle_id][cluster][0] + 0.000001)
            # если bundle_id нет в словаре
            else:
                current_list[cnt, 0] = 0.67
                current_list[cnt, 1] = 0.67

        # если лекция
        else: 
            current_list[cnt, 0] = 0
            current_list[cnt, 1] = 0
            
    df_pd['bundle_id_all_accuracy'] = current_list[:,0].astype(np.float32)
    df_pd['bundle_id_cluster_accuracy'] = current_list[:,1].astype(np.float32)

    return df_pd

In [19]:
# user_question_part_accuracy - точность ответа в разрезе part вопроса по юзеру 
question_part_map = {
    'part_-100_count' : 0,
    'part_-100_count_correct' : 1,
    'part_-100_accuracy' : 2,
    
    'part_1_count' : 3,
    'part_1_count_correct' : 4,
    'part_1_accuracy' : 5,
    
    'part_2_count' : 6,
    'part_2_count_correct' : 7,
    'part_2_accuracy' : 8,
    
    'part_3_count' : 9,
    'part_3_count_correct' : 10,
    'part_3_accuracy' : 11,
    
    'part_4_count' : 12,
    'part_4_count_correct' : 13,
    'part_4_accuracy' : 14,
    
    'part_5_count' : 15,
    'part_5_count_correct' : 16,
    'part_5_accuracy' : 17,
    
    'part_6_count' : 18,
    'part_6_count_correct' : 19,
    'part_6_accuracy' : 20,
    
    'part_7_count' : 21,
    'part_7_count_correct' : 22,
    'part_7_accuracy' : 23
}
question_part_def_accuracy = {
    1 : 0.745,
    2 : 0.709,
    3 : 0.701,
    4 : 0.631,
    5 : 0.610,
    6 : 0.669,
    7 : 0.660,
    -100 : 0.6,
}


def dict_user_question_part_accuracy_get(df_pd, feat_dict):
    current_list = np.zeros(len(df_pd))
    for idx, (user, part_q, content_type_id) in enumerate(df_pd[['user_id', 'part_q', 'content_type_id']].values):
        part_q = int(part_q)  
        # если вопрос
        if content_type_id == 0:
            map_accuracy = question_part_map['part_' + str(part_q) + '_accuracy']
            # если юзер есть в словаре - берем из словаря
            if user in feat_dict:
                current_list[idx] = feat_dict[user][map_accuracy]
            else: # если user нет в словаре - берем дефолт из словаря дефолта по part_q
                current_list[idx] = question_part_def_accuracy[part_q]       
        # если лекция - ставим 0
        else:
            current_list[idx] = 0

    return current_list

def dict_user_question_part_accuracy_update(df_pd, feat_dict, trust_border=10):

    for _, (user, part_q, content_type_id, ans_corr) in enumerate(df_pd[['user_id', 'part_q', 'content_type_id', 'answered_correctly']].values):
        part_q = int(part_q)
        # если вопрос
        if content_type_id == 0:
            
            map_cnt = question_part_map['part_' + str(part_q) + '_count']
            map_cnt_correct = question_part_map['part_' + str(part_q) + '_count_correct']
            map_accuracy = question_part_map['part_' + str(part_q) + '_accuracy']
            
            # если юзер есть в словаре
            if user in feat_dict:
                feat_dict[user][map_cnt] += 1
                feat_dict[user][map_cnt_correct] += ans_corr
                feat_dict[user][map_accuracy] = feat_dict[user][map_cnt_correct] / feat_dict[user][map_cnt]
                # если количество вопросов меньше trust_border - сглаживаем средним по part
                if feat_dict[user][map_cnt] < trust_border:
                    feat_dict[user][map_accuracy] = ((feat_dict[user][map_accuracy] * feat_dict[user][map_cnt] +
                    question_part_def_accuracy[part_q] * (trust_border - feat_dict[user][map_cnt])) / trust_border)

            else: # если user нет в словаре - добавляем
                feat_dict[user] = [0] * len(question_part_map)
                for i in range(1, 7):
                    feat_dict[user][question_part_map['part_' + str(i) + '_accuracy']] = question_part_def_accuracy[i]
                feat_dict[user][map_cnt] += 1
                feat_dict[user][map_cnt_correct] += ans_corr
                feat_dict[user][map_accuracy] = feat_dict[user][map_cnt_correct] / feat_dict[user][map_cnt]
                # сглаживаем средним по part
                feat_dict[user][map_accuracy] = ((feat_dict[user][map_accuracy] * feat_dict[user][map_cnt] +
                question_part_def_accuracy[part_q] * (trust_border - feat_dict[user][map_cnt])) / trust_border)

    return feat_dict

In [20]:
def load_obj(name ):
    with open('../input/riiid-numpy-df-3/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

# Load boostings

In [21]:
from catboost import CatBoostClassifier

cat_model = CatBoostClassifier()
cat_model.load_model('../input/riiid-lgb-v1/cat_arvis_v4.cbm')

import lightgbm as lgb
lgb_model = lgb.Booster(model_file='../input/riiid-lgb-v1/model_lgb_7946_v8_full_data_arvis.txt')

# Some dicts

In [22]:
# импорт словарей
dict_lectures = load_obj('dict_lectures')
dict_questions = load_obj('dict_questions')

dict_question_user_cnt = load_obj('dict_question_user_cnt')
dict_correct_answers_user_cnt = load_obj('dict_correct_answers_user_cnt')
dict_question_explonation_user_cnt = load_obj('dict_question_explonation_user_cnt')
dict_questionid_part_tag12_avgtarget = load_obj('dict_questionid_part_tag12_avgtarget_5')

dict_user_question_attempt_cnt = load_obj('dict_user_question_attempt_cnt')
dict_user_lectures_part = load_obj('dict_user_lectures_part')
dict_user_question_part_accuracy = load_obj('dict_user_question_part_accuracy')
dict_user_l_q_tag_equal = load_obj('dict_user_l_q_tag_equal')

dict_user_slice_accuracy_5 = load_obj('dict_user_slice_accuracy_5')
dict_user_slice_accuracy_20 = load_obj('dict_user_slice_accuracy_20')
dict_user_slice_accuracy_50 = load_obj('dict_user_slice_accuracy_50')
dict_user_slice_accuracy_session_3 = load_obj('dict_user_slice_accuracy_session_3')
dict_user_slice_accuracy_session_12 = load_obj('dict_user_slice_accuracy_session_12')
dict_user_slice_accuracy_session_48 = load_obj('dict_user_slice_accuracy_session_48')
dict_user_timestampsdelta_3 = load_obj('dict_user_timestampsdelta_3')

dict_global_question_tag_accuracy = load_obj('dict_global_question_tag_accuracy')
dict_global_question_tag_accuracy[-100] = 0.64
dict_user_question_tag_accuracy = load_obj('dict_user_question_tag_accuracy')

dict_user_correct_incorrect_timestamp = load_obj('dict_user_correct_incorrect_timestamp')
dict_content_elapsed_time_mean = load_obj('dict_content_elapsed_time_mean')
dict_user_slice_question_time_mean_session = load_obj('dict_user_slice_question_time_mean_session')

dict_user_priorq_expl_types = load_obj('dict_user_priorq_expl_types')

dict_user_lectures_typeof_cnt = load_obj('dict_user_lectures_typeof_cnt')
dict_user_answer_mode_10 = load_obj('dict_user_answer_mode_10')
dict_user_answer_mode_50 = load_obj('dict_user_answer_mode_50')

dict_question_bundle_accuracy = load_obj('dict_question_bundle_accuracy')
dict_user_bundle_cluster = load_obj('dict_user_bundle_cluster')

* 0 - row_id
* 1 - timestamp
* 2 - user_id
* 3 - content_id
* 4 - content_type_id
* 5 - task_container_id
* 6 - prior_question_elapsed_time
* 7 - prior_question_had_explanation
* 8 - prior_group_answers_correct
* 9 - prior_group_responses

In [23]:
features_map = {# input
                'row_id' : 0,
                'timestamp' : 1,
                'user_id' : 2,
                'content_id' : 3,
                'content_type_id' : 4,
                'task_container_id' : 5,
                'prior_question_elapsed_time' : 6,
                'prior_question_had_explanation' : 7,
                'prior_group_answers_correct' : 8,
                'prior_group_responses' : 9,
            # 1. dict_user_timestampsdelta_3
                'prior_question_1_timedelta_min' : 10,
                'prior_lecture_1_timedelta_min' : 11,
                'prior_question_2_timedelta_min' : 12,
                'prior_lecture_2_timedelta_min' : 13,
                'prior_question_3_timedelta_min' : 14,
                'prior_lecture_3_timedelta_min' : 15,
            # 2. скользящая точность по user_id
                'user_slice_accuracy_5' : 16,
                'user_slice_accuracy_20' : 17,
                'user_slice_accuracy_50' : 18,           
                'user_slice_accuracy_session_3' : 19,
                'user_slice_accuracy_session_12' : 20,
                'user_slice_accuracy_session_48' : 21,
            # 3.1. количество вопросов в группе 
                'task_container_freq' : 22,
            # 3.2. порядковый номер вопроса в группе
                'task_container_counter' : 23,
            # 4. количество вопросов по content_id на которые отвечал каждый user_id накопленно (добавляем + обновляем словарь)  
                'user_question_attempt_cnt' : 24,
            # 5. количество лекций по пользователю накопленно
                'lecture_cnt' : 25,
                'lecture_concept_cnt' : 26,
                'lecture_solving_question_cnt' : 27,
            # 6. сколько лекций с part равным part вопроса пользователь прослушал до вопроса (добавляем + обновляем словарь)
                'part_l_q_cnt' : 28,
            # 7. слушал ли юзер лекцию по тегу равную тегу вопроса (количество) (добавляем + обновляем словарь)
                'tag_l_q_equal_cnt' : 29,
            # 8. точность ответа в разрезе part вопроса по юзеру (добавляем)
                'user_question_part_accuracy' : 30,    
            # 9. точность ответа в разрезе tag вопроса по юзеру (добавляем)
                'user_question_tag_accuracy' : 31, 
            # 10. время до последнего правильного и неправильного ответа
                'prior_question_incorrect_timedelta_min' : 32,
                'prior_question_correct_timedelta_min' : 33,
            # 11. среднее время ответа на вопросы за последние N минут; отношение к среднему по content_id
                'user_question_time_mean_n_session' : 34,
                'user_question_time_mean_all_session' : 35,
                'user_question_time_delta_n_session' : 36,
                'user_question_time_delta_all_session' : 37,
            # 12 prior_question_had_explanation counter features
                'user_prior_correct_expl_prc' : 38,
                'user_prior_correct_noexpl_prc' : 39,
                'user_prior_wrong_expl_prc' : 40,
                'user_prior_wrong_noexpl_prc' : 41,
                'user_prior_answer_expl_type' : 42,
            # 13 мода ответа на последние N вопросов == ответу на вопрос
                'user_answer_mode_10' : 43,
                'user_answer_mode_50' : 44,
            # 14 точность по bundle_id вопроса
                'first_bundle_id_cluster' : 45,
                'bundle_id_all_accuracy' : 46,
                'bundle_id_cluster_accuracy' : 47,
            # 15. количество вопросов по пользователю накопленно (добавляем из словаря)          
                'question_user_cnt' : 48,
            # 16. количество правильных ответов по пользователю накопленно (добавляем из словаря)
                'correct_answers_user_cnt' : 49,
            # 17. доля правильных ответов по пользователю накопленно = correct_answers_user_cnt / question_user_cnt
                'correct_answers_user_prc' : 50,
            # 18. количество вопросов с объяснением по пользователю накопленно (добавляем из словаря)
                'prior_question_had_explanation_user_cnt' : 51,
            # 19. доля вопросов с объяснением по пользователю накопленно
                'prior_question_had_explanation_user_prc' : 52,
            # 20. {question_id : [content_id_mean, part, part_mean, tag_1_mean, tag_2_mean, part_tag_1_mean, part_tag_2_mean]}               
                'content_id_mean' : 53,
                'part' : 54,
                'part_mean' : 55,
                'tag_1_mean' : 56,
                'tag_2_mean' : 57,
#                 'part_tag_1_mean' : 35,
#                 'part_tag_2_mean' : 36,                  
#                 'tags_encoded' : 37,   
            # 21. делим точность пользователя на точность вопроса
                'user_to_question_accuracy' : 58, 
            # 22. среднее гармоническое из точности пользователя и точности вопроса
                'hmean_user_content_accuracy' : 59 
               }

dict_questionid_part_tag12_avgtarget_map = {
    'content_id_cnt' : 0,
    'content_correct_cnt' : 1,
    
    'answered_correctly_avg_content_smooth' : 2,
    'part' : 3,
    'answered_correctly_avg_part' : 4,
    'answered_correctly_avg_tag_1' : 5,
    'answered_correctly_avg_tag_2' : 6,
    'answered_correctly_avg_part_tag_1' : 7,
    'answered_correctly_avg_part_tag_2' : 8,
    'tags_encoded' : 9 
}

train_cols_clf = [features_map['prior_question_1_timedelta_min'],
              features_map['prior_question_2_timedelta_min'],
              features_map['prior_question_3_timedelta_min'],
              features_map['prior_lecture_1_timedelta_min'],
              features_map['prior_lecture_2_timedelta_min'],
              features_map['prior_lecture_3_timedelta_min'],
              features_map['task_container_freq'],
              features_map['task_container_counter'],             
              features_map['user_question_attempt_cnt'],
              
              # features_2
              features_map['prior_question_elapsed_time'],
              features_map['prior_question_had_explanation'],
              features_map['question_user_cnt'],
              features_map['correct_answers_user_cnt'],
              features_map['correct_answers_user_prc'],
              features_map['prior_question_had_explanation_user_cnt'],
              features_map['prior_question_had_explanation_user_prc'],
              
              # features_3
              features_map['user_slice_accuracy_5'],
              features_map['user_slice_accuracy_20'],
              features_map['user_slice_accuracy_50'],
              
              features_map['user_slice_accuracy_session_3'],
              features_map['user_slice_accuracy_session_12'],
              features_map['user_slice_accuracy_session_48'],
 
              # features_4
              features_map['lecture_cnt'],
              features_map['lecture_concept_cnt'],
              features_map['lecture_solving_question_cnt'],
              features_map['part_l_q_cnt'],
              features_map['tag_l_q_equal_cnt'], 
              
              # features_5
              features_map['user_question_part_accuracy'],
              
              # features_6
              features_map['user_question_tag_accuracy'],
              
              # features_8
              features_map['prior_question_incorrect_timedelta_min'],
              features_map['prior_question_correct_timedelta_min'],
              features_map['user_question_time_mean_n_session'],
              features_map['user_question_time_mean_all_session'],
              features_map['user_question_time_delta_n_session'],
              features_map['user_question_time_delta_all_session'],
              
              # features_9
              features_map['user_prior_correct_expl_prc'],
              features_map['user_prior_correct_noexpl_prc'],
              features_map['user_prior_wrong_expl_prc'],
              features_map['user_prior_wrong_noexpl_prc'],
              features_map['user_prior_answer_expl_type'],
              
              # features_10
              features_map['user_answer_mode_10'],
              features_map['user_answer_mode_50'],              
              
              # features_11
              features_map['first_bundle_id_cluster'],
              features_map['bundle_id_all_accuracy'],
              features_map['bundle_id_cluster_accuracy'],
              
              # dict
              features_map['content_id_mean'],
              features_map['part'],
              features_map['part_mean'],
              features_map['tag_1_mean'],
              features_map['tag_2_mean'],
              #features_map['part_tag_1_mean'],
              #features_map['part_tag_2_mean'],
              #features_map['tags_encoded'],
              
              # calc
              features_map['user_to_question_accuracy'], 
              features_map['hmean_user_content_accuracy'], 
              
             ]

train_cols_lgb = [features_map['prior_question_1_timedelta_min'],
              features_map['prior_question_2_timedelta_min'],
              features_map['prior_question_3_timedelta_min'],
              features_map['prior_lecture_1_timedelta_min'],
              features_map['prior_lecture_2_timedelta_min'],
              features_map['prior_lecture_3_timedelta_min'],
              features_map['task_container_freq'],
              features_map['task_container_counter'],             
              features_map['user_question_attempt_cnt'],
              
              # features_2
              features_map['prior_question_elapsed_time'],
              features_map['prior_question_had_explanation'],
              features_map['question_user_cnt'],
              features_map['correct_answers_user_cnt'],
              features_map['correct_answers_user_prc'],
              features_map['prior_question_had_explanation_user_cnt'],
              features_map['prior_question_had_explanation_user_prc'],
              
              # features_3
              features_map['user_slice_accuracy_5'],
              features_map['user_slice_accuracy_20'],
              features_map['user_slice_accuracy_50'],
              
              features_map['user_slice_accuracy_session_3'],
              features_map['user_slice_accuracy_session_12'],
              features_map['user_slice_accuracy_session_48'],
 
              # features_4
              features_map['lecture_cnt'],
              features_map['lecture_concept_cnt'],
              features_map['lecture_solving_question_cnt'],
              features_map['part_l_q_cnt'],
              features_map['tag_l_q_equal_cnt'], 
              
              # features_5
              features_map['user_question_part_accuracy'],
              
              # features_6
              features_map['user_question_tag_accuracy'],
              
              # features_8
              features_map['prior_question_incorrect_timedelta_min'],
              features_map['prior_question_correct_timedelta_min'],
              features_map['user_question_time_mean_n_session'],
              features_map['user_question_time_mean_all_session'],
              features_map['user_question_time_delta_n_session'],
              features_map['user_question_time_delta_all_session'],
              
              # features_9
              features_map['user_prior_correct_expl_prc'],
              features_map['user_prior_correct_noexpl_prc'],
              features_map['user_prior_wrong_expl_prc'],
              features_map['user_prior_wrong_noexpl_prc'],
              features_map['user_prior_answer_expl_type'],
              
              # features_10
              features_map['user_answer_mode_10'],
              features_map['user_answer_mode_50'],              
              
              # features_11
              features_map['first_bundle_id_cluster'],
              features_map['bundle_id_all_accuracy'],
              features_map['bundle_id_cluster_accuracy'],
              
              # dict
              features_map['content_id_mean'],
              features_map['part'],
              features_map['part_mean'],
              features_map['tag_1_mean'],
              features_map['tag_2_mean'],
              #features_map['part_tag_1_mean'],
              #features_map['part_tag_2_mean'],
              #features_map['tags_encoded'],
              
              # calc
              features_map['user_to_question_accuracy'], 
              features_map['hmean_user_content_accuracy'], 
              
             ]

In [24]:
idx = 86867031

In [25]:
import riiideducation
env = riiideducation.make_env()
iter_test = env.iter_test()

In [26]:
# for boosting
previous_test_df = pd.DataFrame()

# for saint
prev_test_df = None

for (test_df, sample_prediction_df) in iter_test:
    # make a copy, because preprocessing could be different
    test_df_saint = test_df.copy()
    ## SAINT PART
    if (prev_test_df is not None) & (psutil.virtual_memory().percent < 90):
        prev_test_df['answered_correctly'] = eval(test_df['prior_group_answers_correct'].iloc[0])
        prev_test_df = prev_test_df[prev_test_df.content_type_id == False]
        
        ## lag time
        prev_test_df = feature_time_lag(prev_test_df, time_dict)

        prev_group = prev_test_df[['user_id', 'content_id', 'answered_correctly', 'part', 'prior_question_elapsed_time', 'time_lag', 'prior_question_had_explanation']].groupby('user_id').apply(lambda r: (
            r['content_id'].values,
            r['answered_correctly'].values,
            r['part'].values,
            r['prior_question_elapsed_time'].values,
            r['time_lag'].values,
            r['prior_question_had_explanation'].values))
        
        for prev_user_id in prev_group.index:
            if prev_user_id in group.index:
                group[prev_user_id] = (
                    np.append(group[prev_user_id][0], prev_group[prev_user_id][0])[-MAX_SEQ:], 
                    np.append(group[prev_user_id][1], prev_group[prev_user_id][1])[-MAX_SEQ:],
                    np.append(group[prev_user_id][2], prev_group[prev_user_id][2])[-MAX_SEQ:],
                    np.append(group[prev_user_id][3], prev_group[prev_user_id][3])[-MAX_SEQ:],
                    np.append(group[prev_user_id][4], prev_group[prev_user_id][4])[-MAX_SEQ:],
                    np.append(group[prev_user_id][5], prev_group[prev_user_id][5])[-MAX_SEQ:]
                )
 
            else:
                group[prev_user_id] = (
                    prev_group[prev_user_id][0], 
                    prev_group[prev_user_id][1],
                    prev_group[prev_user_id][2],
                    prev_group[prev_user_id][3],
                    prev_group[prev_user_id][4],
                    prev_group[prev_user_id][5]
                )

            
    ## elapsed time
    test_df_saint.prior_question_elapsed_time = test_df_saint.prior_question_elapsed_time.fillna(0)
    
    ## prior_question_had_explanation
    test_df_saint['prior_question_had_explanation'] = test_df_saint['prior_question_had_explanation'].fillna(value = False).astype(int)
    
    test_df_saint = test_df_saint.merge(questions_df[["question_id","part"]], how = "left",left_on = 'content_id', right_on = 'question_id')  
              
    prev_test_df = test_df_saint.copy()
            
    ## drop lecture
    test_df_saint = test_df_saint[test_df_saint.content_type_id == False]
    
    
    ## lag time
    test_df_saint = feature_time_lag(test_df_saint, time_dict)
  
    test_dataset = TestDataset(group, test_df_saint, n_skill)
    test_dataloader = DataLoader(test_dataset, batch_size=51200, shuffle=False)
    
    outs1 = [] 
    outs2 = []
    outs3 = []
        
    for item in test_dataloader:
        exercise = item[0].to(device).long()
        part = item[1].to(device).long()
        response = item[2].to(device).long()
        elapsed_time = item[3].to(device).long()
        lag_time = item[4].to(device).long()
        pri_exp = item[5].to(device).long()
        
        with torch.no_grad():
            output1 = model1(exercise, part, response, elapsed_time, lag_time, pri_exp)
            output2 = model2(exercise, part, response, elapsed_time, lag_time, pri_exp)
            output3 = model3(exercise, part, response, elapsed_time, lag_time, pri_exp)
        outs1.extend(torch.sigmoid(output1)[:, -1].view(-1).data.cpu().numpy())
        outs2.extend(torch.sigmoid(output2)[:, -1].view(-1).data.cpu().numpy())
        outs3.extend(torch.sigmoid(output3)[:, -1].view(-1).data.cpu().numpy())

    ## Catboost part
    # если previous_test_df существует и не пустой - обновляем словари
    if previous_test_df.shape[0] != 0:
        # get correct answers for previous group
        previous_test_df['answered_correctly'] = eval(test_df['prior_group_answers_correct'].iloc[0])
        previous_test_df['user_answer'] = eval(test_df['prior_group_responses'].iloc[0])
        # оставляем только вопросы
        previous_test_df = previous_test_df[previous_test_df['content_type_id'] == 0]
        previous_test_df['q_counter'] = 1
               
        # 1. количество вопросов по пользователю накопленно - обновляем словарь
        update_dict(previous_test_df, dict_question_user_cnt, 'user_id', 'q_counter')
        # 2. количество правильных ответов по пользователю накопленно - обновляем словарь
        update_dict(previous_test_df, dict_correct_answers_user_cnt, 'user_id', 'answered_correctly')
        # 3. количество вопросов с объяснением по пользователю накопленно - обновляем словарь
        previous_test_df['prior_question_had_explanation'] = previous_test_df['prior_question_had_explanation'].fillna(0).astype(int)
        update_dict(previous_test_df, dict_question_explonation_user_cnt, 'user_id', 'prior_question_had_explanation')
        # 4. количество вопросов по content_id - обновляем словарь
        update_dict(previous_test_df, dict_questionid_part_tag12_avgtarget, 'content_id', 'q_counter', 0)
        # 5. количество правильных ответов по content_id - обновляем словарь
        update_dict(previous_test_df, dict_questionid_part_tag12_avgtarget, 'content_id', 'answered_correctly', 1)
        # 6. сглаженная доля правильных ответов по content_id - обновляем словарь
        answered_correctly_avg_content_global = 0.5 # средняя точность для нового типа вопросов - на шару
        trust_border = 5 # требуемое количество вопросов нового типа для расчета полной статистики без сглаживания
        for c in previous_test_df['content_id'].unique():
            #dict_questionid_part_tag12_avgtarget[c][2] =  dict_questionid_part_tag12_avgtarget[c][1] / dict_questionid_part_tag12_avgtarget[c][0]      
            # несглаженная точность
            answered_correctly_avg_content = dict_questionid_part_tag12_avgtarget[c][1] / dict_questionid_part_tag12_avgtarget[c][0]
            # если количество вопросов больше trust_border - сглаживать не надо, просто считаем точность
            if dict_questionid_part_tag12_avgtarget[c][0] >= trust_border:
                dict_questionid_part_tag12_avgtarget[c][2] =  answered_correctly_avg_content
            # если количество вопросов меньше trust_border - сглаживаем
            else: 
                border_calc_K = np.minimum(dict_questionid_part_tag12_avgtarget[c][0], trust_border)
                border_calc_L = (trust_border - border_calc_K) / trust_border
                dict_questionid_part_tag12_avgtarget[c][2] = (
                    answered_correctly_avg_content * border_calc_K / trust_border +
                    answered_correctly_avg_content_global * border_calc_L
                )        
        
        # 7. user_question_part_accuracy -  точность ответа в разрезе part вопроса по юзеру - обновляем словарь
        dict_user_question_part_accuracy = dict_user_question_part_accuracy_update(previous_test_df, dict_user_question_part_accuracy)
        
        # 8. скользящая точность по user_id - обновляем словарь
        dict_user_slice_accuracy_5 = user_slice_accuracy_n_update(previous_test_df, dict_user_slice_accuracy_5, border=5)
        dict_user_slice_accuracy_20 = user_slice_accuracy_n_update(previous_test_df, dict_user_slice_accuracy_20, border=20)
        dict_user_slice_accuracy_50 = user_slice_accuracy_n_update(previous_test_df, dict_user_slice_accuracy_50, border=50)
        dict_user_slice_accuracy_session_3 = user_slice_accuracy_session_update(previous_test_df, dict_user_slice_accuracy_session_3, session_max_time=3)
        dict_user_slice_accuracy_session_12 = user_slice_accuracy_session_update(previous_test_df, dict_user_slice_accuracy_session_12, session_max_time=12)
        dict_user_slice_accuracy_session_48 = user_slice_accuracy_session_update(previous_test_df, dict_user_slice_accuracy_session_48, session_max_time=48)
        # 9. время до последнего правильного и неправильного ответа - обновляем словарь
        dict_user_correct_incorrect_timestamp = user_correct_incorrect_timestamp_update(previous_test_df, dict_user_correct_incorrect_timestamp)
        # 10. user_question_tag_accuracy -  точность ответа в разрезе part вопроса по юзеру - обновляем словарь
        dict_user_question_tag_accuracy = user_question_tag_accuracy_update(previous_test_df, dict_user_question_tag_accuracy)
        # 11. prior_question_had_explanation -  counter features - обновляем словарь
        dict_user_priorq_expl_types = user_priorq_expl_types_update(previous_test_df, dict_user_priorq_expl_types)
        # 12. user_answer_mode_10/50 - мода ответа на последние N вопросов == номер правильного ответа по content_id - обновляем словарь
        dict_user_answer_mode_10 = user_answer_mode_n_update(previous_test_df, dict_user_answer_mode_10, border=10)
        dict_user_answer_mode_50 = user_answer_mode_n_update(previous_test_df, dict_user_answer_mode_50, border=50)
        # 13. bundle_id_accuracy - точность по bundle_id вопроса - обновляем словарь
        dict_question_bundle_accuracy = question_bundle_accuracy_update(previous_test_df, dict_question_bundle_accuracy)
        
        ### new feature
        update_user_feats(previous_test_df)

        for user_id, answered_correctly, t in zip(previous_test_df['user_id'].values, previous_test_df['answered_correctly'].values, previous_test_df['timestamp'].values):
            if user_id in lt_correct_dict['timestamp']:
                if t == lt_correct_dict['timestamp'][user_id]:
                    lt_correct_dict['last_timestamp_correct_cnt'][user_id] += 1
                    lt_correct_dict['last_timestamp_correct_sum'][user_id] += answered_correctly
                else:
                    lt_correct_dict['timestamp'].update({user_id:t})
                    lt_correct_dict['last_timestamp_correct_cnt'][user_id] = 1
                    lt_correct_dict['last_timestamp_correct_sum'][user_id] = answered_correctly
            else:
                lt_correct_dict['timestamp'].update({user_id:t})
                lt_correct_dict['last_timestamp_correct_cnt'].update({user_id:1})
                lt_correct_dict['last_timestamp_correct_sum'].update({user_id:answered_correctly})
                
            lt_correct_dict['last_timestamp_correct_pct'][user_id] = lt_correct_dict['last_timestamp_correct_sum'][user_id] / lt_correct_dict['last_timestamp_correct_cnt'][user_id]
            
                
            
    # сохраняем копию
    previous_test_df = test_df.copy()
    test_df2 = test_df.copy()
    # если батч пустой - не делаем расчеты
#     if test_df.shape[0] == 0: 
#         test_df['answered_correctly'] = []
#         env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])
#     else:       
    
    ########## РАСЧЕТ ФИЧЕЙ ##########
        
    # заполняем пропуски
    prior_question_elapsed_time_mean = 25452.541

    
    ##### new feature part
    test_df2 = test_df[test_df['content_type_id'] == 0].reset_index(drop=True)
    lt_correct_cnt = np.zeros(len(test_df2), dtype=np.int8)
    lt_correct_sum = np.zeros(len(test_df2), dtype=np.int8)
    lt_correct_pct = np.zeros(len(test_df2), dtype=np.float16)
    test_df2 = add_user_feats_without_update(test_df2)
    test_df2 = add_uq_feats_and_update(test_df2) # new
    test_df2 = lagtime_for_test(test_df2)

    for i, (user_id, t) in enumerate(zip(test_df2['user_id'].values, test_df2['timestamp'].values)):
        if user_id in lt_correct_dict['timestamp']:
            lt_correct_cnt[i] = lt_correct_dict['last_timestamp_correct_cnt'][user_id]
            lt_correct_sum[i] = lt_correct_dict['last_timestamp_correct_sum'][user_id]
            lt_correct_pct[i] = lt_correct_dict['last_timestamp_correct_pct'][user_id]
        else:
            lt_correct_cnt[i] = -1
            lt_correct_sum[i] = -1
            lt_correct_pct[i] = -1

    test_df2['last_timestamp_correct_cnt'] = lt_correct_cnt
    test_df2['last_timestamp_correct_sum'] = lt_correct_sum
    test_df2['last_timestamp_correct_pct'] = lt_correct_pct

    additional_test_feature = test_df2[['last_timestamp_correct_cnt', 'last_timestamp_correct_sum',
   'last_timestamp_correct_pct', 'lag_time', 'lag_time2',
   'lag_time3', 'curr_user_time_diff', 'curr_user_time_diff_mean',
   'curr_user_elapsed_time_diff', 'curr_uq_time_diff']].values  
    
    test_df['prior_question_elapsed_time'] = test_df['prior_question_elapsed_time'].fillna(prior_question_elapsed_time_mean)
    test_df['prior_group_answers_correct'] = 0
    test_df['prior_group_responses'] = 0
    test_df = test_df.replace([np.inf, -np.inf], np.nan)
    test_df = test_df.fillna(0)    
    ##### 1. сколько времени прошло с момента последнего вопроса и лекции: prior_question_timedelta_min, prior_lecture_timedelta_min
    # считаем фичи, заполняем словарь
    test_df = dict_user_timestampsdelta_get_update_3(test_df, dict_user_timestampsdelta_3)

    ##### 2. скользящая точность по user_id
    test_df['user_slice_accuracy_5'] = user_slice_accuracy_n_get(test_df, dict_user_slice_accuracy_5)
    test_df['user_slice_accuracy_20'] = user_slice_accuracy_n_get(test_df, dict_user_slice_accuracy_20)
    test_df['user_slice_accuracy_50'] = user_slice_accuracy_n_get(test_df, dict_user_slice_accuracy_50)
    test_df['user_slice_accuracy_session_3'] = user_slice_accuracy_session_get(test_df, dict_user_slice_accuracy_session_3, session_max_time=3)
    test_df['user_slice_accuracy_session_12'] = user_slice_accuracy_session_get(test_df, dict_user_slice_accuracy_session_12, session_max_time=12)
    test_df['user_slice_accuracy_session_48'] = user_slice_accuracy_session_get(test_df, dict_user_slice_accuracy_session_48, session_max_time=48)

    ##### 3.1. количество вопросов в группе
    test_df['task_container_freq'] = test_df.groupby(['user_id', 'task_container_id'])['task_container_id'].transform('count')
    ##### 3.2. порядковый номер вопроса в группе
    test_df['task_container_counter'] = test_df[['user_id', 'task_container_id', 'content_id']].groupby(['user_id', 'task_container_id'], as_index=False).agg(['cumcount']) + 1     
    ##### 4. количество вопросов по content_id на которые отвечал каждый user_id накопленно (добавляем + обновляем словарь)      
    test_df['user_question_attempt_cnt'] = user_question_attempt_cnt_get_update(test_df, dict_user_question_attempt_cnt).astype(np.int16)
    ##### 5. количество лекций по пользователю накопленно
    test_df['lecture_cnt'] = user_lecture_cnt(test_df, dict_user_lectures_part)
    test_df = user_lectures_typeof_cnt(test_df, dict_user_lectures_typeof_cnt)
    ##### 6. part_l_q_cnt - сколько лекций с part равным part вопроса пользователь прослушал до вопроса (добавляем + обновляем словарь) 
    # добавляем part и tags_list вопроса
    test_df['part_q'] = get_q_l(test_df, dict_questions, 'part')
    test_df['tags_list'] = get_q_l(test_df, dict_questions, 'tags_list')
    # добавляем part и tag лекций
    test_df['part_l'] = get_q_l(test_df, dict_lectures, 'part')
    test_df['tag_l'] = get_q_l(test_df, dict_lectures, 'tag')    
    test_df['part_l_q_cnt'] = user_lectures_part(test_df, dict_user_lectures_part).astype(np.int16)

    ##### 7. tag_l_q_equal_cnt - слушал ли юзер лекцию по тегу равную тегу вопроса (количество) (добавляем + обновляем словарь)
    test_df['tag_l_q_equal_cnt'] = user_l_q_tag_equal(test_df, dict_user_l_q_tag_equal).astype(np.int16)

    ##### 8. user_question_part_accuracy -  точность ответа в разрезе part вопроса по юзеру (добавляем)
    test_df['user_question_part_accuracy'] = dict_user_question_part_accuracy_get(test_df, dict_user_question_part_accuracy)
    ##### 9. user_question_tag_accuracy -  точность ответа в разрезе tag вопроса по юзеру (добавляем)
    test_df['user_question_tag_accuracy'] = user_question_tag_accuracy_get(test_df, dict_user_question_tag_accuracy)
    ##### 10. время до последнего правильного и неправильного ответа
    test_df = user_correct_incorrect_timestamp_get(test_df, dict_user_correct_incorrect_timestamp)
    ##### 11. среднее время ответа на вопросы за последние N минут; отношение к среднему по content_id
    test_df = user_slice_question_time_mean_session(test_df, dict_user_slice_question_time_mean_session)
    ##### 12. prior_question_had_explanation -  counter features
    test_df = user_priorq_expl_types_get(test_df, dict_user_priorq_expl_types)
    ##### 13. user_answer_mode_10/50 - мода ответа на последние N вопросов == номер правильного ответа по content_id
    test_df['user_answer_mode_10'] = user_answer_mode_n_get(test_df, dict_user_answer_mode_10, border=10)
    test_df['user_answer_mode_50'] = user_answer_mode_n_get(test_df, dict_user_answer_mode_50, border=50)
    ##### 14 точность по bundle_id вопроса
    # тянем bundle_id
    test_df = question_bundle_id_get(test_df, dict_questions)
    # кластер юзера по первому bundle_id - расчет фичи + заполнение словаря
    test_df = user_bundle_cluster_get_update(test_df, dict_user_bundle_cluster)
    # тянем точность по кластеру юзера по первому bundle_id
    test_df = question_bundle_accuracy_get(test_df, dict_question_bundle_accuracy)
    
    # сохраняем part_q, tags_list, task_container_freq, first_bundle_id_cluster в previous_test_df
    previous_test_df['part_q'] = test_df['part_q'].values
    previous_test_df['tags_list'] = test_df['tags_list'].values
    previous_test_df['task_container_freq'] = test_df['task_container_freq'].values
    previous_test_df['bundle_id'] = test_df['bundle_id'].values
    previous_test_df['first_bundle_id_cluster'] = test_df['first_bundle_id_cluster'].values
    # удаляем временные столбцы
    test_df.drop(columns=['part_l', 'part_q', 'tag_l', 'tags_list', 'bundle_id'], inplace=True)
    # оставляем только вопросы
    test_df = test_df[test_df['content_type_id'] == 0]
    
    # если фрейм не пустой (кейс когда в батче только лекция)
    if test_df.shape[0] > 0:
    
        # преобразуем в numpy
        np_test_df = test_df.to_numpy(dtype=np.float64, na_value=0)
        col_idx = features_map['user_id']

        ##### 15. количество вопросов по пользователю накопленно (добавляем из словаря)
        np_test_df = add_feats_from_dict(np_test_df, dict_question_user_cnt, col_idx)    
        ##### 16. количество правильных ответов по пользователю накопленно (добавляем из словаря)
        np_test_df = add_feats_from_dict(np_test_df, dict_correct_answers_user_cnt, col_idx)
        ##### 17. доля правильных ответов по пользователю накопленно
        # делим количество правильных ответов на количество вопросов, если знаменатель = 0, тогда 0
        col_numerator = features_map['correct_answers_user_cnt']
        col_denominator = features_map['question_user_cnt']
        np_test_df = np.c_[ np_test_df, np.divide(np_test_df[:,col_numerator], np_test_df[:,col_denominator], 
                                                  out=np.zeros_like(np_test_df[:,col_denominator]), 
                                                  where=np_test_df[:,col_denominator]!=0) ]

        ##### 18. количество вопросов с объяснением по пользователю накопленно (добавляем из словаря)
        np_test_df = add_feats_from_dict(np_test_df, dict_question_explonation_user_cnt, col_idx)
        ##### 19. доля вопросов с объяснением по пользователю накопленно
        # делим количество вопросов с объяснением на количество вопросов, если знаменатель = 0, тогда 0
        col_numerator = features_map['prior_question_had_explanation_user_cnt']
        col_denominator = features_map['question_user_cnt']
        np_test_df = np.c_[ np_test_df, np.divide(np_test_df[:,col_numerator], np_test_df[:,col_denominator], 
                                                  out=np.zeros_like(np_test_df[:,col_denominator]), 
                                                  where=np_test_df[:,col_denominator]!=0) ]        
        ##### 20. {question_id : [content_id_mean, part, part_mean, tag_1_mean, tag_2_mean, part_tag_1_mean, part_tag_2_mean]}
        for i in [dict_questionid_part_tag12_avgtarget_map['answered_correctly_avg_content_smooth'],
                  dict_questionid_part_tag12_avgtarget_map['part'],
                  dict_questionid_part_tag12_avgtarget_map['answered_correctly_avg_part'],
                  dict_questionid_part_tag12_avgtarget_map['answered_correctly_avg_tag_1'],
                  dict_questionid_part_tag12_avgtarget_map['answered_correctly_avg_tag_2'],
    #               dict_questionid_part_tag12_avgtarget_map['answered_correctly_avg_part_tag_1'],
    #               dict_questionid_part_tag12_avgtarget_map['answered_correctly_avg_part_tag_2'],
    #               dict_questionid_part_tag12_avgtarget_map['tags_encoded'],
                 ]:
            np_test_df = add_feats_from_dict_got_new_user(np_test_df, dict_questionid_part_tag12_avgtarget, col_idx=features_map['content_id'], col_dict=i)

        ##### 21. user_to_question_accuracy
        # делим точность пользователя на точность вопроса
        # если знаменатель = 0, тогда 0
        col_numerator = features_map['correct_answers_user_prc']
        col_denominator = features_map['content_id_mean']
        np_test_df = np.c_[ np_test_df, np.divide(np_test_df[:,col_numerator], np_test_df[:,col_denominator], out=np.zeros_like(np_test_df[:,col_denominator]), where=np_test_df[:,col_denominator]!=0) ]

        ##### 22. hmean_user_content_accuracy
        # среднее гармоническое из точности пользователя и точности вопроса
        user_acc = features_map['correct_answers_user_prc']
        question_acc = features_map['content_id_mean']
        np_test_df = np.c_[ np_test_df, 2*(np_test_df[:,user_acc] * np_test_df[:,question_acc])/(np_test_df[:,user_acc] + np_test_df[:,question_acc]) ]
        

        pred_feature = np_test_df[:,train_cols_clf]
        pred_feature = np.concatenate((pred_feature, additional_test_feature), axis=1)
        ########## GET MODEL PREDICT ##########
# dict_keys(['timestamp', 'last_timestamp_correct_cnt', 'last_timestamp_correct_sum', 'last_timestamp_correct_pct'])


        pred = pd.DataFrame()
        pred['score_lgbm'] = lgb_model.predict(pred_feature)
        pred['score_cat'] = cat_model.predict_proba(pred_feature)[:,1]
        pred['score_saint1'] = np.array(outs1)
        pred['score_saint2'] = np.array(outs2)
        pred['score_saint3'] = np.array(outs3)
        
        test_df['answered_correctly'] = (c1*pred['score_lgbm'] + c1_2 * pred['score_cat'] + c2*pred['score_saint1'] + c3*pred['score_saint2']+ c4*pred['score_saint3']).values

        
    else:
        test_df['answered_correctly'] = np.array([], dtype = np.float32)

    env.predict(test_df[['row_id', 'answered_correctly']])

In [27]:
pred

,score_lgbm,score_cat,score_saint1,score_saint2,score_saint3
0,0.478748,0.612215,0.527656,0.401783,0.502617
1,0.641462,0.554773,0.533524,0.586037,0.603591
2,0.618951,0.615681,0.418596,0.445075,0.419311
3,0.962143,0.964624,0.963496,0.982206,0.966599
4,0.611812,0.641657,0.483666,0.466230,0.485718
5,0.506414,0.559490,0.337574,0.522273,0.329698
6,0.945686,0.934640,0.905524,0.946444,0.926528
7,0.785686,0.822789,0.865399,0.832248,0.898405
8,0.544445,0.559445,0.435156,0.500816,0.516480
9,0.888108,0.904807,0.807320,0.697412,0.705825


In [28]:
0.25 * 0.7, 0.25 * 0.3

(0.175, 0.075)